<a href="https://colab.research.google.com/github/Meguazy/HACK_4DS/blob/main/notebooks_fra/HACK4DS_Day1_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Meguazy/HACK_4DS.git

Cloning into 'HACK_4DS'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 106 (delta 44), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (106/106), 11.92 MiB | 4.68 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [2]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


In [3]:
%cd /content/HACK_4DS

/content/HACK_4DS


# Initial

In [4]:
!pip install lightgbm imbalanced-learn

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt

In [215]:
df_test = pd.read_csv("data2/test_set.csv", sep=";", quotechar="\"", decimal=",")

In [216]:
df_train = pd.read_csv("data2/train_set.csv", sep=";", quotechar="\"", decimal=",")

# Preprocessing

In [214]:
external_score_ver03_dictionary = {
    "MISSING":None,
    "A":14,
    "B":13,
    "C":12,
    "D":11,
    "E":10,
    "F":9,
    "G":8,
    "H":7,
    "I":6,
    "L":5,
    "M":4,
    "N":3,
    "O":2,
    "P":1
}

df_train = df_train.replace({"external_score_ver03": external_score_ver03_dictionary})
df_test = df_test.replace({"external_score_ver03": external_score_ver03_dictionary})

In [217]:
# Concatenate df_train and df_test
df_combined = pd.concat([df_train, df_test], axis=0)

# List of columns to encode
columns_to_encode = ['province', 'juridical_form', 'industry_sector', 'region', 'geo_area', 'decision_date']

# Instantiate LabelEncoder
label_encoders = {}

# Encode each column iteratively
for column in columns_to_encode:
    # Instantiate LabelEncoder for the current column
    label_encoders[column] = LabelEncoder()

    # Fit label encoder on combined data
    label_encoders[column].fit(df_combined[column])

    # Transform both training and testing data
    df_train[column] = label_encoders[column].transform(df_train[column])
    df_test[column] = label_encoders[column].transform(df_test[column])

In [218]:
class_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'late_payment_score',
       'external_score_late_payment_integrated', 'external_score_moderate',
       'external_score_adverse', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']

reg_columns = ['decision_date', 'company_ID', 'external_score_ver01',
       'external_score_ver02', 'external_score_ver03', 'age', 'province',
       'juridical_form', 'industry_sector', 'gross_margin_ratio',
       'core_income_ratio', 'cash_asset_ratio',
       'consolidated_liabilities_ratio', 'tangible_assets_ratio', 'revenues',
       'cr_available', 'region', 'geo_area', 'last_statement_age',
       'overrun_freq_a_revoca_autoliquidanti',
       'avg_tension_a_revoca_autoliquidanti',
       'std_tension_a_revoca_autoliquidanti',
       'max_tension_a_revoca_autoliquidanti',
       'last_tension_a_revoca_autoliquidanti',
       'avg_rel_used_a_revoca_autoliquidanti',
       'std_rel_used_a_revoca_autoliquidanti',
       'max_rel_used_a_revoca_autoliquidanti',
       'last_rel_used_a_revoca_autoliquidanti', 'overrun_freq_a_scadenza',
       'avg_rel_used_a_scadenza', 'std_rel_used_a_scadenza',
       'max_rel_used_a_scadenza', 'last_rel_used_a_scadenza',
       'avg_count_enti_affidanti', 'std_count_enti_affidanti',
       'max_count_enti_affidanti', 'last_count_enti_affidanti',
       'avg_count_numero_prima_info', 'std_count_numero_prima_info',
       'max_count_numero_prima_info', 'last_count_numero_prima_info']


# Classificator

In [219]:
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(df_train[class_columns], df_train.target, test_size=0.15, random_state=42)

train_data = lgb.Dataset(X_train_class, label=y_train_class)

# Setting up parameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05,
    'random_state': 42,
    'num_iterations': 5000,
    'is_unbalance': True,
    'max_bin': 10,
    'num_leaves': 50,
    'n_estimators': 200,
    'verbose': -1
}

# Training the model
bst = lgb.train(params, train_data)

# Making predictions
y_pred_class = bst.predict(X_test_class, num_iteration=5000)

# Converting probabilities to binary predictions
y_pred_binary = [1 if pred > 0.4132 else 0 for pred in y_pred_class]

# Calculating F1 score
f1 = f1_score(y_test_class, y_pred_binary)
print("F1 score:", f1)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


F1 score: 0.6055149127743388


# Classification Submission

In [190]:
# Making predictions
submission_class = bst.predict(df_test[class_columns], num_iteration=5000)

# Converting probabilities to binary predictions
submission_class_binary = [1 if pred > 0.4132 else 0 for pred in submission_class]

submission = pd.DataFrame(submission_class_binary, columns=["label"])

submission.to_csv("file.csv", index=False)